In [1]:
import pdb, math
import matplotlib.pyplot as plt
import numpy as np
import torch
import pdb
import torch.nn as nn
plt.rcParams["figure.figsize"] = (12,12)

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class SDF(nn.Module):
    def __init__(self):
        super(SDF, self).__init__()
        self.fc1 = nn.Linear(515, 512)
        self.drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        self.fc5 = nn.Linear(512, 512)     
        self.fc6 = nn.Linear(512, 1)
        
    def forward(self, x):
#         pdb.set_trace()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [1]:
import numpy as np
import os
# coords = np.load('coords.npy')
# SDFs = np.load('SDF.npy')

coords = [np.load(f'/home/alex/data/3dc/{x}coords.npy') for x in range(100)]
SDFs = [np.load(f'/home/alex/data/3dc/{x}SDF.npy') for x in range(100)]

coords = np.concatenate(coords)
SDFs = np.concatenate(SDFs)

In [5]:
# idxes = [i for i, s in enumerate(SDFs) if s<20]

In [6]:
from torch.utils.data import Dataset, DataLoader

class SDFDataset(Dataset):

    def __init__(self,coords,SDFs):
        self.coords = coords
        self.SDFs = SDFs

    def __len__(self):
        return len(self.SDFs)

    def __getitem__(self, idx):
        
        coord = torch.from_numpy(self.coords[idx])
        sdf = torch.from_numpy(np.array(self.SDFs[idx])).unsqueeze(0)

#         pdb.set_trace()
        sample = {'coords': coord, 'sdf': sdf}

        return sample

In [7]:
import random
from torch.utils import data
dataset = SDFDataset(coords,SDFs)

idx = list(range(len(coords)))
random.shuffle(idx)  # in-place shuffle the indices to facilitate random splitting
train_idx = idx[:int(len(coords)*0.8)]
test_idx = idx[int(len(coords)*0.8):]

train_set = data.Subset(dataset, train_idx)
test_set = data.Subset(dataset, test_idx)

train_dataloader = DataLoader(train_set, batch_size=256,
                        shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_set, batch_size=256,
                        shuffle=True, num_workers=4)

In [8]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, sample in enumerate(train_loader):
#         pdb.set_trace()
        data, target = sample['coords'].to(device), sample['sdf'].to(device)
        optimizer.zero_grad()
        output = model(data)
#         pdb.set_trace()
        
        loss = F.l1_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, device, test_loader, optimizer, epoch, best_loss=1):
    model.eval()
    losses = []
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample['coords'].to(device), sample['sdf'].to(device)
            output = model(data)
            loss = F.l1_loss(output, target)
            losses.append(loss)
    #     pdb.set_trace()
        loss = sum(losses)/len(losses)
        print(f'test loss:{loss}')
        if loss<best_loss:
            torch.save(model.state_dict(), 'sdf_with_latent.torch')
            best_loss = loss
        return best_loss

In [9]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
model = SDF().to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.00001)
best_loss = 1

for epoch in range(1, 50 + 1):
    train(model, device, train_dataloader, optimizer, epoch)
    best_loss = test(model, device, test_dataloader, optimizer, epoch, best_loss)

Train Epoch: 1 [0/1561122 (0%)]	Loss: 5.439217
Train Epoch: 1 [256000/1561122 (16%)]	Loss: 0.347641
Train Epoch: 1 [512000/1561122 (33%)]	Loss: 0.236828
Train Epoch: 1 [768000/1561122 (49%)]	Loss: 0.130061
Train Epoch: 1 [1024000/1561122 (66%)]	Loss: 0.127078
Train Epoch: 1 [1280000/1561122 (82%)]	Loss: 0.108003
Train Epoch: 1 [1536000/1561122 (98%)]	Loss: 0.095373
test loss:0.12245526164770126
Train Epoch: 2 [0/1561122 (0%)]	Loss: 0.121494
Train Epoch: 2 [256000/1561122 (16%)]	Loss: 0.088297
Train Epoch: 2 [512000/1561122 (33%)]	Loss: 0.093667
Train Epoch: 2 [768000/1561122 (49%)]	Loss: 0.088872
Train Epoch: 2 [1024000/1561122 (66%)]	Loss: 0.082109
Train Epoch: 2 [1280000/1561122 (82%)]	Loss: 0.088363
Train Epoch: 2 [1536000/1561122 (98%)]	Loss: 0.085791
test loss:0.08147022873163223
Train Epoch: 3 [0/1561122 (0%)]	Loss: 0.069396
Train Epoch: 3 [256000/1561122 (16%)]	Loss: 0.074865
Train Epoch: 3 [512000/1561122 (33%)]	Loss: 0.084982
Train Epoch: 3 [768000/1561122 (49%)]	Loss: 0.07846

In [20]:
model.load_state_dict(torch.load('SDF.torch'))

<All keys matched successfully>

In [25]:
import numpy as np
model = model.eval()
model(torch.from_numpy(np.array([2.3454492 , -0.96645486,  8.859472])).unsqueeze(0).float())

tensor([[4.0132]], grad_fn=<AddmmBackward>)

In [22]:
for i, x in enumerate(SDFs):
    if x<-0.2: break

In [23]:
SDFs[i]

4.0081353

In [24]:
coords[i]

array([ 2.3454492 , -0.96645486,  8.859472  ], dtype=float32)